In [1]:
!pip install keras_bert
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp37-none-any.whl size=34144 sha256=1bd82b45f3b75223837335cd2cb44d052fe374e34c4f8f04d0bc00341a12dba7
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp37-none-any.whl size=12942 sha256=a3606dfe091650adff513cde15a25e2284846d17ebbe36dcef076563cb5b93f9
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp37-none-any.whl size=7554 sha256=b29ed87ecc3572f365c211ccc47036726f30321860d5172243194acc940447aa
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp37-none-any.whl size=15611 sha256=76faacf97a713548136297e9fd074d7fe3e902626e580c0231bf5913b776dcd1
  Stor

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!cd gdrive/My' 'Drive/Colab' 'Notebooks
!ls gdrive/My' 'Drive/Colab' 'Notebooks
import os
os.chdir("gdrive/My Drive/Colab Notebooks")

Mounted at /content/gdrive
 bert.ipynb		        modelvc_colab.model.trainables.syn1neg.npy
 bertmodel		        modelvc_colab.model.wv.vectors.npy
 bert_trained.png	        modelvc.model
 bot_model3.py		        modelvc.model.syn1neg.npy
 bot_model_fin2.py	        modelvc.model.wv.vectors.npy
 bot_model_final.py	        movie_conversations.txt
 bot_model_fin.py	        movie_lines.txt
 bot_models.py		        pre2_run.ipynb
 bot_train_savefile.py	        pre2_savefile.ipynb
 conv.txt		        pre.ipynb
'Copy of pre2_savefile.ipynb'   __pycache__
'Copy of pre.ipynb'	        questions-word2s.txt
 data_2.json		        questions-words.txt
 datapre.py		        td.txt
 embeddings_colab.kv	        test.txt
 embeddings.kv		        uncased_L-2_H-128_A-2
 fc_net_git.py		        Untitled
 fc_net.py		        Untitled0.ipynb
 layers			        Untitled1.ipynb
 model			        Untitled2.ipynb
 model_plot.png		        Untitled3.ipynb
 modelvc_colab.model	        w2v.txt


In [11]:
def movieinput():
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    import re
    import time
    lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
    conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    convs = []
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
        convs.append(_line.split(','))
    questions = []
    answers = []
    corpus = []
    for conv in convs:
        for i in range(len(conv) - 1):
            cot = [id2line[conv[i]], id2line[conv[i + 1]]]
            corpus.append(cot)
    return corpus

In [12]:
corpus = movieinput()

In [13]:
corpus[1:10]

[["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part.  Please.'],
 ['Not the hacking and gagging and spitting part.  Please.',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"],
 ["You're asking me out.  That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.'],
 ['Cameron.',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."],
 ["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
  'Seems like she could get a date easy enough...'],
 ['Why?',
  'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.'],
 ['Unsolved mystery.  She used to be really popular when she started 

In [25]:
model_path = 'uncased_L-2_H-128_A-2'
from keras_bert import extract_embeddings
embeddings = extract_embeddings(model_path, corpus[1])

In [26]:
len(embeddings)

2

In [27]:
len(embeddings[0])

21

In [28]:
len(embeddings[0][0])

128

In [30]:
embeddings

[array([[-1.5220592 ,  0.50088394, -3.4219606 , ...,  0.45740902,
         -0.45696044,  0.97163993],
        [-1.6494675 ,  0.9875897 ,  0.03026113, ..., -1.3667037 ,
         -0.8411627 ,  0.67170614],
        [-1.804443  ,  0.34176323,  0.13734329, ..., -1.4380163 ,
         -1.0494798 ,  0.85824823],
        ...,
        [-0.82620764,  0.6032084 , -0.03958166, ..., -0.8831636 ,
         -0.55166745, -0.12525228],
        [-0.6985612 ,  0.483358  , -0.04208551, ..., -0.68918407,
         -1.1334566 , -0.27376273],
        [-1.965588  ,  0.33203375,  0.18568626, ..., -0.84344494,
         -0.65154445,  1.2338033 ]], dtype=float32),
 array([[-1.8759732 ,  0.13834034, -2.9566934 , ...,  0.05538473,
         -0.8409858 ,  0.8340823 ],
        [-1.3315489 ,  1.3347914 ,  0.02588546, ..., -1.0139276 ,
         -0.9998584 ,  1.6313146 ],
        [-2.2180653 ,  0.8426324 ,  0.10486621, ..., -1.3958938 ,
         -1.4867004 ,  0.7733764 ],
        ...,
        [-1.4714628 ,  1.2466362 ,  0.0

In [203]:
class MySeq2Seq(object):

    def __init__(self, max_seq_len = 10, word_vec_dim = 128, input_file='./segment_result_lined.3000000.pair.less'):
        self.max_seq_len = max_seq_len
        self.word_vec_dim = word_vec_dim
        self.input_file = input_file
        self.setup = False
        self.vocabsize = 0
    
    def generate_trainig_data(self):

        import re
        if not self.setup:
          lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
          conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
          id2line = {}
          for line in lines:
            _line = line.split(' +++$+++ ')
            if len(_line) == 5:
                id2line[_line[0]] = _line[4]
          convs = []
          for line in conv_lines[:-1]:
            _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
            convs.append(_line.split(','))
          questions = []
          answers = []


          for conv in convs:
            for i in range(len(conv) - 1):
                questions.append(id2line[conv[i]])
                answers.append(id2line[conv[i + 1]])
          question = [re.sub(r'[^A-Za-z0-9 -]+', '', word.lower())  for word in questions]
          answer = [re.sub(r'[^A-Za-z0-9 -]+', '', word.lower())  for word in answers]
          #self.setup = True
        x_data = []
        y_data = []
        t_data = []
        self.vocabsize = len(question)
        for i in range(min(len(question),limit)):
        #for i in range(100):
            question_seq = question[i]
            answer_seq = answer[i]
            [question_seq, answer_seq] = extract_embeddings(model_path, [question_seq, answer_seq])
            question_seq = question_seq.tolist()
            answer_seq = answer_seq.tolist()
            if len(question_seq) < self.max_seq_len and len(answer_seq) < self.max_seq_len and i<limit:
                print(i);
                sequence_x = list(question_seq) + [dmin*np.ones(self.word_vec_dim)] * (self.max_seq_len-len(question_seq))
                sequence_y2 = [dmax*np.ones(self.word_vec_dim)] + answer_seq
                sequence_t = answer_seq + [-dmax*np.ones(self.word_vec_dim)]
                sequence_y2 = sequence_y2 + [dmin * np.ones(self.word_vec_dim)] * (self.max_seq_len - len(answer_seq) - 1)
                sequence_t = sequence_t + [dmin * np.ones(self.word_vec_dim)] * (
                            self.max_seq_len - len(answer_seq) - 1)
                x_data.append(sequence_x)
                y_data.append(sequence_y2)
                t_data.append(sequence_t)
               # ttt11 = question[i]
               # ttt2 = answer[i]
               # ttt2
                #print "right answer"
                #for w in answer_seq:
                #    (match_word, max_cos) = vector2word(w)
                #    if len(match_word)>0:
                #        print match_word, vector_sqrtlen(w)

        return np.array(x_data), np.array(y_data),np.array(t_data)

    def generate_test(self,str):
        from gensim.models import Word2Vec, KeyedVectors
        data = KeyedVectors.load(wordvecname)
        ##############
        datatt = len(data['you'])
        self.word_vec_dim = datatt
        import re
        str =re.sub(r'[^A-Za-z0-9 -]+', '', str.lower())
        question_seq = extract_embeddings(model_path, [str])
        question_seq = question_seq
        X = []
        y = []
        tmp1 = question_seq
        [tmp1] = tmp1
        tmp1 = tmp1.tolist()
        tmp1 = [np.array(tmpp) for tmpp in tmp1]
        if len(tmp1) < self.max_seq_len:
                #sequence_x =  list(tmp1) + [np.zeros(word_vec_dim)] * (self.max_seq_len - len(tmp1))
                sequence_x = list(tmp1) + [0* np.ones(word_vec_dim)] * (
                            self.max_seq_len - len(question_seq))
                # sequence_xy = list(tmp1) + [np.zeros(word_vec_dim)] * (self.max_seq_len - len(tmp1))
                sequence_y = [np.zeros(word_vec_dim)] * self.max_seq_len
                X.append(sequence_x)
        return np.array(X)

    def define_models(self,n_input, n_output, n_units):
        import keras
        from tensorflow.keras import layers
        # define training encoder

        encoder_inputs = layers.Input(shape=(None, n_input))

     #   encoder_batch = BatchNormalization()
     #   encoder_inputt =  encoder_batch(encoder_inputs)

        encoder =   layers.LSTM(n_units, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]
       # encoder_outputs, state_h, state_c = encoder(encoder_inputs,initial_state=encoder_states)
       # encoder_states = [state_h, state_c]
        # define training decoder

        decoder_inputs = layers.Input(shape=(None, n_output))
        decoder_lstm = layers.LSTM(n_units, return_sequences=True, return_state=True)


        decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
      #  decoder_dense = layers.Dense(n_output)
        decoder_dense  = keras.Sequential(
    [
        layers.Dense(n_output, activation="tanh", name="layer1"),
        layers.Dense(n_output, activation="tanh", name="layer2"),
        layers.Dense(n_output, activation="tanh", name="layer3"),
      #  layers.Dense(n_output, activation="tanh", name="layer4"),
     #   layers.Dense(n_output, activation="tanh", name="layer5"),
    #    layers.Dense(n_output, activation="tanh", name="layer6"),
     #   layers.Dense(n_output, activation="tanh", name="layer7"),
     #   layers.Dense(n_output, activation="tanh", name="layer8"),
    #    layers.Dense(n_output, activation="tanh", name="layer9"),

        layers.Dense(n_output, name="layerfin"),
    ]
    )

        decoder_outputs = decoder_dense(decoder_outputs)
        model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
        # define inference encoder
        encoder_model = keras.Model(encoder_inputs, encoder_states)
        # define inference decoder


        decoder_state_input_h = layers.Input(shape=(n_units,))
        decoder_state_input_c = layers.Input(shape=(n_units,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
        # return all models
        return model, encoder_model, decoder_model




    def train(self):
        import matplotlib.pyplot as plt
        trainXY, trainY, traint = self.generate_trainig_data()
        model, infenc, infdec = self.define_models(word_vec_dim,word_vec_dim,word_vec_dim)
       # model = self.model(feed_previous=False)
        optimizer = keras.optimizers.Adam(lr=0.1)
        model.compile(optimizer=optimizer, loss='cosine_similarity', metrics=['accuracy'])
        history = model.fit([trainXY, trainY],traint,epochs=ep)
        model.save('./model/bot')
        infenc.save('./model/encoder')
        infdec.save('./model/decoder')
        print('saved')
        plt.plot(history.history['acc'])
        plt.title('0.1 lr')
        plt.ion()
        plt.pause(0.1)
        plt.show()
      #   model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
        # list all data in history
        return model

    def train2(self):
        import matplotlib.pyplot as plt
        trainXY, trainY, traint = self.generate_trainig_data()
        model, infenc, infdec = self.define_models(word_vec_dim, word_vec_dim, word_vec_dim)
        # model = self.model(feed_previous=False)
        optimizer = keras.optimizers.Adam(lr=0.0005)
        model.compile(optimizer=optimizer, loss='cosine_similarity', metrics=[soft_acc])
        history = model.fit([trainXY, trainY], traint, epochs=ep,shuffle=True )
        model.save('./model/botfinb')
        infenc.save('./model/encoderfinb')
        infdec.save('./model/decoderfinb')
        print('saved')
        plt.plot(history.history['loss'])
        plt.title('model14')
        plt.show()
        #   model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
        # list all data in history
        return model

    def load(self):
        dependencies = {
            'soft_acc': soft_acc
        }
        from keras.models import load_model
        #model = self.model(feed_previous=True)
        self.model= load_model('./model/botfinb',custom_objects=dependencies )
        self.infenc= load_model('./model/encoderfinb',custom_objects=dependencies )
        self.infdec= load_model('./model/decoderfinb',custom_objects=dependencies )
        print(self.model.summary())
        from keras.utils.vis_utils import plot_model
        plot_model(self.model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


    def mypredict(self, source, n_steps, cardinality):
        # encode
        infenc = self.infenc
        infdec = self.infdec
        state = infenc.predict(source)
        # start of sequence input
        target_seq = np.array([1.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
        # collect predictions
        output = list()
        for t in range(n_steps):
            # predict next char
            yhat, h, c = infdec.predict([target_seq] + state)
            # store prediction
            output.append(yhat[0, 0, :])
            # update state
            state = [h, c]
            # update target sequence
            target_seq = yhat
            #target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
        return np.array(output)

In [109]:
max_w = 10
float_size = 4
wordvecname = "embeddings_colab.kv"
from gensim.models import Word2Vec, KeyedVectors
word_vector_dict = KeyedVectors.load(wordvecname)
word_vec_dim = 128
max_seq_len = 10
word_set = {}
limit = 10000
ep = 500
wordvc =  Word2Vec.load("modelvc_colab.model")
data = KeyedVectors.load(wordvecname)
dmax = 1#np.max(data.vectors) # 1 : value for <pad> and <eos>
dmin = 0#np.min(data.vectors) # 0 : value for <go>


In [86]:
def init_seq(input_file):
    """读取切好词的文本文件，加载全部词序列
    """


    ##############
    datatt = len(word_vector_dict['you'])
    word_vec_dim = datatt

    file_object = open(input_file, 'r')
    vocab_dict = {}
    while True:
        question_seq = []
        answer_seq = []
        line = file_object.readline()
        if line:
            line_pair = line.split('|')
            line_question = line_pair[0]
            line_answer = line_pair[1]
            for word in line_question.decode('utf-8').split(' '):
                if word_vector_dict.has_key(word):
                    question_seq.append(word_vector_dict[word])
            for word in line_answer.decode('utf-8').split(' '):
                if word_vector_dict.has_key(word):
                    answer_seq.append(word_vector_dict[word])
        else:
            break
        question_seqs.append(question_seq)
        answer_seqs.append(answer_seq)
    file_object.close()

def vector_sqrtlen(vector):
    len = 0
    for item in vector:
        len += item * item
    len = math.sqrt(len)
    return len

def vector_cosine(v1, v2):
    if len(v1) != len(v2):
        sys.exit(1)
    sqrtlen1 = vector_sqrtlen(v1)
    sqrtlen2 = vector_sqrtlen(v2)
    value = 0
    for item1, item2 in zip(v1, v2):
        value += item1 * item2
    return value / (sqrtlen1*sqrtlen2)


def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true*10), K.round(y_pred*10)))


In [89]:
import os

pretrained_path = 'uncased_L-2_H-128_A-2'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [90]:
os.environ['TF_KERAS'] = '1'

In [91]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}

In [92]:
def vector2word(vector):
    max_cos = -10000
    match_word = ''
    for word in token_dict_inv:
        v = token_dict_inv[word]
        cosine = vector_cosine(vector, v)
        if cosine > max_cos:
            max_cos = cosine
            match_word = word
    return (match_word, max_cos)

In [194]:
my_seq2seq = MySeq2Seq(word_vec_dim=word_vec_dim, max_seq_len=max_seq_len)

In [94]:
from keras import backend as K

In [204]:
my_seq2seq2 = MySeq2Seq(word_vec_dim=word_vec_dim, max_seq_len=max_seq_len)

In [205]:
my_seq2seq2.load()

Model: "model_22855"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 128), (None, 131584      input_13[0][0]                   
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, None, 128),  131584      input_14[0][0]                   
                                                                 lstm_6[0][1]           

In [206]:

testX1 ='Have fun tonight?'
testX2 ="you have my word,  as a gentleman"
testX3 ='do you listen to this crap'
testX4 ='i figured youd get to the good stuff eventually'
testX5 ='what good stuff'
testX6 ='wow'
testX7 ='she okay'
testX8 ='It\'s more'
testX9 ='did you change your hair'
testX10 ='Let go!'
testX = [testX1,testX2,testX3,testX4,testX5,testX6,testX8,testX7,testX9,testX10]

In [164]:
str = 'Have fun tonight?'
import re
str =re.sub(r'[^A-Za-z0-9 -]+', '', str.lower())
question_seq = extract_embeddings(model_path, [str])
question_seq = question_seq
X = []
y = []
tmp1 = question_seq
[tmp1] = tmp1
tmp1 = tmp1.tolist()
tmp1 = [np.array(tmpp) for tmpp in tmp1]
if len(tmp1) < max_seq_len:
                #sequence_x =  list(tmp1) + [np.zeros(word_vec_dim)] * (self.max_seq_len - len(tmp1))
                sequence_x = list(tmp1) + [0* np.ones(word_vec_dim)] * (
                            max_seq_len - len(question_seq))

In [155]:
len(tmp1)

5

In [156]:
len(tmp1[0])

128

In [157]:
kz = [0* np.ones(word_vec_dim)] * (
                            max_seq_len - len(question_seq))

In [160]:
np.shape(kz)

(9, 128)

In [165]:
sequence_x

[array([-1.29485559e+00, -2.14129090e-01, -2.85094738e+00, -1.98327637e+00,
         3.89618695e-01,  1.15067780e+00,  1.46932125e-01,  2.49609613e+00,
        -1.96652126e+00,  2.63208635e-02,  7.09880233e-01,  3.83294761e-01,
        -1.33765066e+00,  5.82899213e-01,  2.14976740e+00, -1.32491839e+00,
         1.04852080e+00, -6.81312740e-01, -1.67227352e+00,  8.06618094e-01,
        -1.06836390e+00, -5.75590491e-01,  2.68891263e+00,  5.33423185e-01,
         7.89784729e-01, -9.70524430e-01,  1.10562837e+00,  7.55183876e-01,
         1.18872985e-01, -1.43736649e+00, -1.06693912e+00, -1.26755941e+00,
        -1.69631362e+00,  1.28626812e+00,  5.24123728e-01, -7.57753313e-01,
        -7.60401189e-01, -6.29025400e-01, -1.76945245e+00, -1.77697229e+00,
         1.77378666e+00, -1.83434081e+00,  9.23717558e-01, -7.45762646e-01,
        -4.65715528e-02, -1.65405869e+00, -5.93829632e-01, -5.01224518e-01,
         2.91868865e-01, -3.67331594e-01,  5.28004885e-01,  1.97760594e+00,
         5.8

In [207]:
 testXX = my_seq2seq2.generate_test(testX1)

In [200]:
nb =[na.tolist() for na in testXX[0]]

In [201]:
nb = np.array(nb)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [220]:
len(nb[5])

200

In [211]:
predict = my_seq2seq2.mypredict(testXX, max_seq_len, word_vec_dim)
print("\n")
print(testX1)

for w in predict:
        # print(w)
        (match_word, max_cos, eosdist) = vector2word(w)
        if(match_word[0][1] <  0.6):
           break
        print(match_word, vector_sqrtlen(w), "  ", eosdist)



Have fun tonight?


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [212]:
predict0 = predict[0]

In [213]:
predict0

array([-0.69180506,  0.4026589 , -1.939438  , -1.1189653 , -0.21768863,
        0.5073965 ,  0.07968546,  0.57965124, -0.43340355,  0.7392013 ,
        0.67940027, -0.3604221 , -1.4776    ,  0.3892886 ,  1.7091602 ,
       -0.7188203 , -0.31905043,  0.05204627, -1.4533765 ,  0.7278389 ,
       -0.17760351, -0.08913042,  1.653678  ,  0.14793953,  0.47093737,
       -0.39816988,  0.4972838 ,  0.6214813 ,  0.25860184, -0.01979379,
       -0.52762425, -1.3060529 , -0.77410907,  0.5636999 ,  0.42371267,
       -1.216814  ,  0.13049692, -0.26267973, -1.1138179 , -1.1489426 ,
        1.1169072 , -0.892715  ,  0.75676763, -0.8865153 , -0.23759688,
       -1.5581158 , -0.21461837, -0.6857289 ,  0.36782655,  0.13229069,
        0.04508353,  0.7291424 ,  0.3099703 , -0.05992098,  0.7172866 ,
       -0.6836921 ,  0.24160744, -0.06818961, -0.5716443 ,  0.9411679 ,
        1.2535756 , -1.0820894 , -0.7825355 , -0.344931  , -0.4709722 ,
        0.4364276 , -0.0944005 ,  0.32162768,  0.8698002 ,  0.48

In [216]:
max_cos = -10000
match_word = ''
vector = predict0

In [219]:
for word in token_dict_inv:
        v = token_dict_inv[word]
        cosine = vector_cosine(vector, v)
        if cosine > max_cos:
            max_cos = cosine
            match_word = word


/bin/bash: line 0: fg: no job control


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


TypeError: ignored